# Data preprocessing

In this notebook we merge sex and marital status information from the ```compas.db``` file, provided by ProPublica in their 2016 article, to the information from the ```features_before_on.csv``` file, provided by Rudin et al. in their 2019 article, which is a version of the ProPublica dataset that is supplemented with probation data they purchased. Then, we further combine it with the recidivism and violent recidivism outcomes, as computed by Rudin et al. based on their expanded dataset. The result is then saved to ```rstats.csv```. 

In order to generate the ```features_before_on.csv``` file, download the repository at https://github.com/beauCoker/age_of_unfairness, and place the required data files as instructed in their readme. Then, run the ```db2csv.r``` file and then the ```Table_construction.rmd``` file. This creates a variable titled ```features_before_on``` and ```outcomes``` – save these ```csv``` files and use in this notebook. 

### Imports

In [214]:
import pandas as pd
import sqlite3

pd.set_option('future.no_silent_downcasting', True)

### Load and preprocess the ProPublica frames.

In [215]:
# Replace 'your_database.db' with the path to your .db file
database_path = 'data/compas.db'

# Establish a connection to the database
conn = sqlite3.connect(database_path)

# Create a cursor object
cursor = conn.cursor()

# Extract the people and compas tables from the database. 
people = pd.read_sql_query(f"SELECT * FROM people", conn)
compas = pd.read_sql_query(f"SELECT * FROM compas", conn)

# Close the connection
conn.close()

# Rename the 'id' column in people to 'person_id', for merging with other tables.
people = people.rename(columns={'id': 'person_id'})

# Select the columns of interest. 
people = people[['person_id', 'sex']]
compas = compas[['person_id', 'marital_status']]

# Change 'sex' column values to 1, for 'Male', and 0, for 'Female'.
people["sex"] = people["sex"].replace({'Male' : 1, 'Female' : 0})
people.rename(columns={"sex": "male"}, inplace=True)

# Remove rows from compas that have an "Unknown" marital_status.
compas = compas[compas["marital_status"] != "Unknown"]

# Change 'marital_status' column values to 1 and 0, binarily indicating relationship status. 
r_dict = {
    'Single' : 0,   
    'Divorced' : 0,
    'Significant Other' : 1,
    'Married' : 1,
    'Separated' : 0,
    'Widowed' : 0
}
compas["marital_status"] = compas["marital_status"].replace(r_dict)
compas.rename(columns={"marital_status": "partnered"}, inplace=True)

# Each person has at least three rows in the compas df, 
# corresponding to the three different risk scores. 
# Since some people were scored multiple times, some 
# persons have more than three rows, and in some instances
# their partnered status changed between scorings. 
# We will simply pick the first value of the variable.
# Since there are few rows with multiple scorings, and even
# fewer with changing partnered status inbetween scorings,
# this has a negligible impact. 
desc = compas.groupby('person_id')
d = [[key, item["partnered"].iloc[0]] for key, item in desc]
compas = pd.DataFrame(d, columns=['person_id', 'partnered'])

### Load and preprocess the frames from Rudin et al.

In [216]:
features_before_on = pd.read_csv('data/rudin/features_before_on.csv')
outcomes = pd.read_csv('data/rudin/outcomes.csv')

# Rename the risk score columns. 
rename_cols = {
       'Risk of Failure to Appear_decile_score' : 'fta_ds',
       'Risk of Failure to Appear_raw_score' : 'fta_rs',
       'Risk of Recidivism_decile_score' : 'grecid_ds', 
       'Risk of Recidivism_raw_score' : 'grecid_rs',
       'Risk of Violence_decile_score' : 'vrecid_ds', 
       'Risk of Violence_raw_score' : 'vrecid_rs', 
}
features_before_on.rename(columns=rename_cols, inplace=True)

# Drop the temporal information. 
dropcols = ['screening_date', 'first_offense_date',
       'current_offense_date', 'before_cutoff_date']
features_before_on.drop(dropcols, axis='columns', inplace=True)

# Drop the screening date column. 
outcomes = outcomes.drop(['screening_date'], axis='columns')

# Some people have multiple screenings, and so have multiple outcomes associated. 
# We simply pick the first of these label occurrances to use for our analysis. 
desc = outcomes.groupby('person_id')
d = [[key, item["recid"].iloc[0], item["recid_violent"].iloc[0]] for key, item in desc]
outcomes = pd.DataFrame(d, columns=['person_id', 'recid', 'recid_violent'])

### Merge the frames, do some final processing, and write to output ```csv```. 

In [220]:
# Merge the frames
mdf = features_before_on.merge(people, on='person_id', how='left')
mdf = mdf.merge(compas, on='person_id', how='left')
mdf = mdf.merge(outcomes, on='person_id', how='left')

# Drop the 'p_famviol_arrest', as it always equals 0.
mdf = mdf.drop(['p_famviol_arrest'], axis='columns')

# Remove rows with negative decile scores.
mdf = mdf[mdf["fta_ds"] > -1]
mdf = mdf[mdf["grecid_ds"] > -1]
mdf = mdf[mdf["vrecid_ds"] > -1]

# Remove rows with p_age_first_offense equal to 0.
mdf = mdf[mdf["p_age_first_offense"] > 0]

# Drop rows with missing values. 
mdf = mdf.dropna()

# Cast the value types in all columns in cs to int.
cs = [c for c in mdf.columns if c not in ['grecid_rs', 'vrecid_rs']]
for c in mdf.columns:
    if c not in ['grecid_rs', 'vrecid_rs']:
        mdf[c] = mdf[c].astype(int)

# Drop the index column.
mdf = mdf.drop(['person_id'], axis='columns')

# Write mdf to a csv file in the 'data' subfolder. 
mdf.to_csv('data/rscores.csv', index=False)
print("Finished writing frames to csv files.")

Finished writing frames to csv files.
